In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.156, 0.075, 0.099, 0.101],
    [0.081, 0.072, 0.081, 0.093],
    [0.055, 0.086, 0.067, 0.090],
    [0.055, 0.070, 0.080, 0.080],
    [0.067, 0.081, 0.238, 0.071],
    [0.243, 0.057, 0.117, 0.062],
    [0.032, 0.128, 0.017, 0.131],
    [0.023, 0.088, 0.022, 0.134],
    [0.030, 0.119, 0.112, 0.095],
    [0.081, 0.092, 0.125, 0.081],
    [0.011, 0.131, 0.042, 0.063]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

# NORMALIZATION FUNCTION (min-max normalization)
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# CODAS METHOD
def codas(matrix, weights, criteria_types, sites, criteria, tau=0.02):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Negative-ideal solution (worst for each criterion)
    neg_ideal = np.min(weighted_matrix, axis=0)

    # Euclidean and Taxicab distances from negative-ideal
    euclidean = np.sqrt(((weighted_matrix - neg_ideal) ** 2).sum(axis=1))
    taxicab = np.abs(weighted_matrix - neg_ideal).sum(axis=1)

    # Assessment score
    # If difference in Euclidean distance between two alternatives < tau, use Taxicab distance as tie-breaker
    scores = np.zeros(len(sites))
    for i in range(len(sites)):
        count = 0
        for j in range(len(sites)):
            if i == j:
                continue
            diff = euclidean[i] - euclidean[j]
            if diff > tau:
                count += 1
            elif abs(diff) <= tau:
                if taxicab[i] > taxicab[j]:
                    count += 1
        scores[i] = count

    # Alternatively, for ranking, you can use the Euclidean distance directly (higher is better)
    df = pd.DataFrame({
        'Euclidean': euclidean,
        'Taxicab': taxicab,
        'CODAS Score': euclidean,  # For simple ranking, use Euclidean distance
        'Rank': rankdata(-euclidean, method='min')
    }, index=sites).sort_values('Rank')

    return df

result_df = codas(matrix, weights, criteria_types, sites, criteria)
print(result_df)


        Euclidean   Taxicab  CODAS Score  Rank
Site5    0.413578  0.568219     0.413578     1
Site11   0.353574  0.395249     0.353574     2
Site9    0.339596  0.484736     0.339596     3
Site7    0.337341  0.361988     0.337341     4
Site6    0.309849  0.431550     0.309849     5
Site10   0.265047  0.435149     0.265047     6
Site1    0.231709  0.391102     0.231709     7
Site3    0.170630  0.275482     0.170630     8
Site2    0.154481  0.262556     0.154481     9
Site8    0.147819  0.171534     0.147819    10
Site4    0.136592  0.222392     0.136592    11
